In [5]:
import os
import weakref
from dataclasses import dataclass
from itertools import takewhile
from typing import Dict, List

import numpy as np
import pandas as pd
from numba import njit
from numpy.typing import ArrayLike

class Node:
    def __init__(self, u_id: int, neighbors: ArrayLike, skip_unique: False = False):
        self._u_id = u_id
        # produces sorted array
        self.neighbors = np.unique(neighbors) if not skip_unique else neighbors
        
    @property
    def u_id(self):
        return self._u_id
    
    @property
    def deg(self):
        return len(self.neighbors)
    
    def subgraph(self, other_nodes: np.ndarray):
        return Node(self.u_id, get_intersection(self.neighbors, other_nodes), skip_unique=True)
    

class Network:
    def __init__(self, path: str):
        if not os.path.isfile(path):
            raise OSError("wrong graph path")
        
        with open(path) as f:
            skip_rows = len(list(
                takewhile(lambda s: s.startswith("%"), f)
            ))
                
        self.data = pd.read_csv(path, sep=r'\s+', header=None,
                                names=["fr", "to", "weight", "timestamp"], skiprows=skip_rows)\
                        .drop(columns=["weight"])\
                        .sort_values(by="timestamp")
        
        self.edges = np.sort(self.data[["fr", "to"]].values)
        self.total_nodes = self.edges.max()
        self.timestamps = self.data.timestamp.values

    def __str__(self):
        ans = "from\tto\tweight\ttimestamp\n"

        for key in sorted(self.__graph.keys()):
            node = self.__graph[key]

            for j in node.edges_to:
                ans += f"{node.u_id}\t{j.node.u_id}\n"

        return ans


class Graph:
    def __init__(self, network: Network, edges: np.ndarray, nodes: dict[int, Node]):
        self.network = network
        self.edges = edges
        self.edges_set: np.ndarray = np.unique(self.edges, axis=0)
        self.nodes = nodes
    
    def get_subgraph(self, nodes_ids: ArrayLike) -> "Graph":
        nodes_ids = np.unique(nodes_ids)
        new_nodes = {u_id : self.nodes[u_id].subgraph(nodes_ids)
                     for u_id in nodes_ids}
        new_edges = np.array([
            [u_id, neighbor] for u_id, node in new_nodes.items() for neighbor in node.neighbors
        ])
        
        return Graph(self.network, new_edges, new_nodes)
    
    @property
    def density(self):
        if len(self.nodes) > 1:
            return 2 * len(self.edges_set) / len(self.nodes) / (len(self.nodes) - 1) 
        return np.nan

class StaticGraph(Graph):
    def __init__(self, timestamps, quantile_end, *args, **kwargs):
        self.timestamps = timestamps
        self.quantile_end = quantile_end
        super().__init__(*args, **kwargs)
        
    @staticmethod
    def from_time_slice(network, quantile_end, quantile_start=0) -> "Graph":
        assert 0 <= quantile_start <= quantile_end <= 1, "Incorrect quantiles"
        
        timestamps = network.timestamps
        left, right = np.quantile(timestamps, [quantile_start, quantile_end])
        mask = (left <= timestamps) & (timestamps <= right)
        edges = network.edges[mask]
        timestamps = timestamps[mask]
        
        undirected = np.vstack([edges, edges[:, ::-1]])
        adj_lists = pd.DataFrame(undirected, columns=["v1", "v2"])\
            .groupby("v1")\
            .v2.apply(np.array)
        
        nodes = {u_id : Node(u_id,np.empty(0, dtype=int)) for u_id in np.arange(1, network.total_nodes + 1)}
        nodes |= {u_id : Node(u_id, neighbors) for u_id, neighbors in adj_lists.items()}
        
        return StaticGraph(timestamps, quantile_end, network, edges, nodes)

In [2]:
def get_all_pairs(current_graph: StaticGraph, future_graph: StaticGraph):
    has_link = []
    no_link = []
    
    visited = np.zeros(len(current_graph.nodes) + 1, dtype=bool)
    for node in current_graph.nodes.values():
        if node.deg <= 1:
            continue
        
        visited[np.concatenate([current_graph.nodes[neighbor_id].neighbors
                                for neighbor_id in node.neighbors])] = True
        visited[node.u_id] = False
        intersection = get_intersection(future_graph.nodes[node.u_id].neighbors, visited.nonzero()[0]).astype(int)
        has_link.append(np.vstack([np.full_like(intersection, node.u_id), intersection]))
        visited[intersection] = False
        
        no_link_t = visited.nonzero()[0]
        no_link.append(np.vstack([np.full_like(no_link_t, node.u_id), no_link_t]))
        visited[no_link_t] = False
    has_link, no_link = [np.hstack(arrs).T for arrs in [has_link, no_link]]
    return has_link, no_link

def get_train_set(graph: StaticGraph, seed: int = 42, size: int = 10_000):
    rng = np.random.RandomState(seed=seed)
    
    future_graph = StaticGraph.from_time_slice(graph.network, 1, graph.quantile_end)
    has_link, no_link = get_all_pairs(graph, future_graph)
    train_edges = np.hstack([
        np.vstack([
            has_link[rng.choice(len(has_link), size)],
            no_link[rng.choice(len(no_link), size)]
        ]),
        np.repeat([1, 0], size).reshape(-1, 1)
    ])
    return train_edges

In [3]:
network = Network("data/opsahl-ucsocial/out.opsahl-ucsocial")
time_quantile = .8

graph = StaticGraph.from_time_slice(network, time_quantile)

In [6]:
@njit
def get_static_features_for_pair(first_neighbors, second_neighbors, degrees):
    intersection = get_intersection(first_neighbors, second_neighbors)
    first_deg, second_deg = first_neighbors.size, second_neighbors.size
    return np.array([
        intersection.size, # CN
        (1 / np.log(degrees[intersection])).sum(), # AA
        intersection.size / (first_deg + second_deg - intersection.size), ## JC
        first_deg * second_deg # PA
    ])

In [7]:
def compute_static_features(graph: StaticGraph, edges: np.ndarray):
    ids = list(graph.nodes.keys())
    degrees_raw = [node.deg for node in graph.nodes.values()]
    
    degrees = np.zeros(graph.network.total_nodes + 1)
    degrees[ids] = degrees_raw
    
    features = np.vstack([
        get_static_features_for_pair(
            graph.nodes[first_id].neighbors, graph.nodes[second_id].neighbors,
            degrees
        ) for first_id, second_id, *_ in edges
    ])
    
    return features

In [35]:
@njit
def _feat(weight):
    return np.array([
            *np.quantile(weight, [0, .25, .5, .75, 1]),
            weight.sum(),
            weight.mean(),
            weight.var()
    ])

@njit
def _post_event_agg(time: np.ndarray, t_min: float, t_max: float, lower_bound: float):
    dt = (time - t_min) / (t_max - t_min)
    weights = lower_bound + (1 - lower_bound) * np.vstack((
        dt,
        (np.exp(3 * dt) - 1) / (np.exp(3) - 1),
        np.sqrt(dt)
    ))
    return np.concatenate((_feat(weights[0]), _feat(weights[1]), _feat(weights[2])))

In [46]:
EPS = 1e-6

def _get_wtfs(indices: list[tuple], df, computed: dict[tuple[int, int], np.ndarray],
             t_max: float, t_min: float, lower_bound: float):
    for index in indices:
        if index[0] > index[1]:
            index = index[::-1]
            
        if index not in computed:
            computed[index] = computed[index[::-1]] = \
                _post_event_agg(df[index], t_max, t_min, lower_bound)
            
    return np.vstack(list(map(computed.get, indices)))

def _sum_outgoing_wtf(node_id, **params):
    edges = [(node_id, neighbor_id) for neighbor_id in graph.nodes[node_id].neighbors]
    return sum(_get_wtfs(indices=edges, **params), start=np.zeros(24))

def _get_temporal_features_for_pair(graph, node_1_id, node_2_id, **params):
    intersection = get_intersection(graph.nodes[node_1_id].neighbors, graph.nodes[node_2_id].neighbors)
    # print([(node_1_id, inters) for inters in intersection])
    # print([(node_2_id, inters) for inters in intersection])
    wtf_sums = _get_wtfs(indices=[(node_1_id, inters) for inters in intersection], **params) + \
               _get_wtfs(indices=[(node_2_id, inters) for inters in intersection], **params)
    
    intersection_sum = np.vstack([_sum_outgoing_wtf(node_id=inters, **params)
                                  for inters in intersection])
    wtf_sum_node_1, wtf_sum_node_2  = [_sum_outgoing_wtf(node_id, **params) 
                                       for node_id in [node_1_id, node_2_id]]
    
    return np.concatenate([
        (wtf_sums / np.log(1 + EPS + intersection_sum)).sum(axis=0), # AA
        wtf_sums.sum(axis=0), # CN
        wtf_sums.sum(axis=0) / (wtf_sum_node_1 + wtf_sum_node_2 + EPS), # JC
        wtf_sum_node_1 * wtf_sum_node_2, # PA
    ])

In [49]:
t = compute_temporal_features(graph, train_set, .2)

 71%|█████████████████████████████████████████████████████▎                     | 14231/20000 [00:06<00:02, 2637.00it/s]

KeyboardInterrupt: 

 71%|█████████████████████████████████████████████████████▎                     | 14231/20000 [00:19<00:02, 2637.00it/s]

In [43]:
np.isnan(t).sum()

204

In [48]:
from tqdm import tqdm
def compute_temporal_features(graph: StaticGraph, edges: np.ndarray, lower_bound):
    grouped = pd.DataFrame({"v1": graph.edges[:, 0], "v2": graph.edges[:, 1], "time": graph.timestamps})\
        .groupby(["v1", "v2"])\
        ["time"].apply(np.array)
    
    computed: dict[tuple[int, int], np.ndarray] = {}
    
    params = dict(
        df=grouped,
        computed=computed,
        t_min=graph.timestamps.min(),
        t_max=graph.timestamps.max(),
        lower_bound=lower_bound
    )
    
    features = np.vstack([
        _get_temporal_features_for_pair(graph, first_id, second_id, **params)
        for first_id, second_id, *_ in tqdm(edges)
    ])
    
    return features

In [ ]:
post_event_agg(np.array([1, 2, 3, 4]), 0, 10, .2)

In [14]:


@njit
def get_intersection(first, second):
    i = j = k = 0
    buffer = np.empty(min(first.size, second.size), dtype=first.dtype)
    while i < first.size and j < second.size:
        if first[i] == second[j]:
            buffer[k] = first[i]
            k += 1
            i += 1
            j += 1
        elif first[i] < second[j]:
            i += 1
        else: 
            j += 1
    return buffer[:k]

In [15]:
num_vertices = len(graph.nodes)
num_edges = len(graph.edges_set)
density = 2 * num_edges / num_vertices / (num_vertices - 1)

In [16]:
from queue import deque

def get_connected_comps(graph: Graph) -> list[list[int]]:
    visited = {u_id : False for u_id in graph.nodes}
    queue = deque(graph.nodes)
    conn_comps: list[list[int]] = []
    
    for root_u_id in graph.nodes.keys():
        if visited[root_u_id]:
            continue
            
        queue = deque([root_u_id])
        conn_comps.append([])
        
        while queue:
            u_id = queue.popleft()
            conn_comps[-1].append(u_id)
            if not visited[u_id]:
                for neighbor_id in graph.nodes[u_id].neighbors:
                    if not visited[neighbor_id]:
                        queue.append(neighbor_id)
                        visited[neighbor_id] = True
    return conn_comps

In [17]:
conn_comps = get_connected_comps(graph)
conn_comp = graph.get_subgraph(max(conn_comps, key=len))
max_conn_comp_fraction =  len(conn_comp.nodes) / num_vertices


In [18]:
from scipy.stats import pearsonr 

def get_avg_cluster_coeff(graph: Graph) -> float:
    return np.nanmean([graph.get_subgraph(node.neighbors).density
                       for node in graph.nodes.values()])

def get_deg_assortivity(graph: Graph) -> float:
    degs = np.vectorize(lambda node_id: graph.nodes[node_id].deg)(graph.edges_set)
    return pearsonr(degs[:, 0], degs[:, 1]).statistic

In [ ]:
from itertools import combinations

In [19]:
train_set = get_train_set(graph)

In [ ]:
second_a = get_features(graph, .2);

In [ ]:
static = get_static_features(graph)

In [ ]:
import pickle
with open("data/opsahl-ucsocial.pickle", "wb") as f:
    pickle.dump({
        "edges_labels": train_set,
        "static": static[train_set[:, 0], train_set[:, 1]],
        "second_a": second_a[train_set[:, 0], train_set[:, 1]]
    }, f)

In [ ]:
t = static[train_set[:, 0], train_set[:, 1]][:, 0]

In [ ]:
import seaborn as sns
sns.histplot(x=np.log(t))